# Import nessesary library

In [ ]:
!pip install vncorenlp

In [ ]:
!pip install vncorenlp transformers

In [ ]:
import pandas as pd
import cv2
from csv import reader
import re
import os
from datetime import datetime
import dateutil.parser
import traceback
import time
import requests
import numpy as np
from random import randint
import json
import pickle
import nltk.data
from sklearn.svm import LinearSVC
from gensim import corpora, matutils
from sklearn.metrics import classification_report
import sklearn
from vncorenlp import VnCoreNLP

# Preprocess Post Message

In [ ]:
import regex as re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)


def vn_word_to_telex_type(word):
    dau_cau = 0
    new_word = ''
    for char in word:
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            new_word += char
            continue
        if y != 0:
            dau_cau = y
        new_word += bang_nguyen_am[x][-1]
    new_word += bang_ky_tu_dau[dau_cau]
    return new_word


def vn_sentence_to_telex_type(sentence):
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = vn_word_to_telex_type(word)
    return ' '.join(words)


def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.

        """
    sentence = sentence
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

In [ ]:
def remove_url(text):
    url_regex = r'<URL>'
    text = re.sub(url_regex, "URL_TAG", text)
    return text
  
def remove_punctuation(text):
    punc_regex = r'([!,?;.\*\"\[\]\&\'\#\(\^\`\~\|\{\}\)\%])\1+|([!,?;.\*\"\[\]\&\'\#\(\^\`\~\|\{\}\)\%])\1+'
    corrected = str(text)
    corrected = re.sub(punc_regex, r'\1', corrected)
    return corrected

def remove_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF" 
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f" 
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


EMAIL = re.compile(r"([\w0-9_\.-:]+)(@)([\d\w\.-]+)(\.)([\w\.]{1,9})")
PHONE = re.compile(r"(09|01[2|6|8|9]) + ([0-9]{8})\b")
MENTION = re.compile(r"@.+?:")
NUMBER =  re.compile(r"\d+.?\d*")
DATETIME = '\d{1,2}\s?[/-]\s?\d{1,2}\s?[/-]\s?\d{4}'

RE_HTML_TAG = re.compile(r'<[^>]+>')
RE_CLEAR_1 = re.compile(r'[^_<>\s\{Latin}]')
RE_CLEAR_2 = re.compile(r'__+')
RE_CLEAR_3 = re.compile(r'\s+')


def replace_common_token(text):
    text = remove_whitespace(text)
    text = re.sub(EMAIL, ' EMAIL_TAG ', text)
    text = re.sub(PHONE, ' PHONE_TAG ', text)
    text = re.sub(MENTION, ' MENTION_TAG ', text)
    return text


def remove_character(text):
    text = re.sub(':v', '', text)
    text = re.sub(':D', '', text)
    text = re.sub(':3', '', text)
    text = re.sub(':\(', '', text)
    text = re.sub(':\)', '', text)
    text = re.sub(':\)\)', '', text)
    text = re.sub(':\(\(', '', text)
    text = re.sub(':\:', '', text)
    return text


def remove_html_tag(text):
    return re.sub(RE_HTML_TAG, ' HTML_TAG ', text)

In [ ]:
def clear_sub(text, tokenize=True):
    text = remove_url(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)
    text = re.sub('&.{3,4};', ' ', str(text))
    text = convert_unicode(text)
    text = replace_common_token(text)
    text = remove_character(text)
    text = chuan_hoa_dau_cau_tieng_viet(text)
    return text.strip()

In [ ]:
rdrsegmenter = VnCoreNLP("/content/gdrive/My Drive/VLSP/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
def preprocess(text):
    try :
        text = clear_sub(text)  
        sents = rdrsegmenter.tokenize(text)
        text = ' '.join([' '.join(sent) for sent in sents])
    except:
        text = ''
        print("xitj")
    return text

In [ ]:
path_train = "path_to_dataset_train"
def build_dataset(path):
    datas = []
    labels = []
    df = pd.read_csv(path)

    df['post_message'] = df['post_message'].apply(preprocess)
    for data in df.values:
        try:
            labels.append(int(data[7]))
        except:
            print(data) 
    return list(df['post_message']), labels    

train_text, train_label = build_dataset(path_train)

# Build Model

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
import torch.nn as nn
from transformers import RobertaModel, BertPreTrainedModel
import torch
import argparse
from transformers import RobertaConfig, AdamW
from torch.nn import CrossEntropyLoss, MSELoss

In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.15, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [ ]:
class RobertaClassificationHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        # self.dropout = nn.Dropout(0.2)
        # self.dense1 = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout1 = nn.Dropout(0.1)
        self.dense2 = nn.Linear(config.hidden_size * 12, config.hidden_size)
        self.dropout2 = nn.Dropout(0.2)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features[:, -1, :]  # take <s> token (equiv. to [CLS])
        # x = self.dropout(x)
        # x = self.dense1(x)
        # x = torch.tanh(x)
        x = self.dropout1(x) # 0.9178
        x = self.dense2(x)
        x = torch.tanh(x)
        x = self.dropout2(x)
        x = self.out_proj(x)
        return x

class RobertaTweetClassification(BertPreTrainedModel):
    config_class = RobertaConfig
    base_model_prefix = "roberta"

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.roberta = RobertaModel(config)
        self.classifier = RobertaClassificationHead(config)
        
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
        sequence_output = torch.cat(outputs[2][1:13], -1)
        
        logits = self.classifier(sequence_output)

        outputs = (logits,) + outputs[2:]
        if labels is not None:
            loss_fct = LabelSmoothingLoss(classes=2, smoothing=0.12) # CrossEntropyLoss() # LabelSmoothingLoss(classes=2, smoothing=0.3)
            loss = loss_fct(logits.view(-1, self.num_labels), labels)
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    y_scores = preds[:, 1].flatten()
    roc_auc = roc_auc_score(labels_flat, y_scores)
    F1_score = f1_score(pred_flat, labels_flat)
    return accuracy_score(pred_flat, labels_flat), F1_score, roc_auc

# Training

In [ ]:
from scipy.special import softmax
import random
import time
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaModel
import torch
import argparse
from transformers import RobertaConfig, AdamW
from transformers import get_linear_schedule_with_warmup, get_constant_schedule
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

device = 'cuda'
loss_values = []

kf = KFold(n_splits = 8)

for fold, (train_index, valid_index) in enumerate(kf.split(train_text, train_label)):

    epochs = 15

    # Roberta
    config = RobertaConfig.from_pretrained(
      "vinai/phobert-base", from_tf=False, num_labels = 2, output_hidden_states=True,
    )
    BERTweet = RobertaTweetClassification.from_pretrained(
      "vinai/phobert-base",
      config=config
    )

    param_optimizer = list(BERTweet.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0005}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)
    BERTweet.to(device)
    frozen = True
    params = list(BERTweet.named_parameters())
    for p in params[:-2]:
        p[1].requires_grad = False 
  
    scheduler0 = get_constant_schedule(optimizer)

    print(train_index)
    train_texts = np.array(train_text)[train_index]
    valid_texts = np.array(train_text)[valid_index]
    train_labels = np.array(train_label)[train_index]
    valid_labels = np.array(train_label)[valid_index]
    train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
    valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)

    train_ids = torch.tensor(train_encodings['input_ids'])
    train_labels = torch.tensor(train_labels)
    train_masks = torch.tensor(train_encodings['attention_mask'])

    valid_ids = torch.tensor(valid_encodings['input_ids'])
    valid_labels = torch.tensor(valid_labels)
    valid_masks = torch.tensor(valid_encodings['attention_mask'])

    train_data = TensorDataset(train_ids, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

    valid_data = TensorDataset(valid_ids, valid_masks, valid_labels)
    valid_sampler = SequentialSampler(valid_data)
    valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=32)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                      num_warmup_steps = 20,
                                      num_training_steps = total_steps)
    roc_max = 0
    for epoch_i in range(0, epochs):
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        t0 = time.time()
        total_loss = 0
        BERTweet.train()
        train_accuracy = 0
        nb_train_steps = 0
        train_f1 = 0

        if epoch_i > 0 and frozen:
            params = list(BERTweet.named_parameters())
            for p in params[:-2]:
              p[1].requires_grad = True

            frozen = False
            del scheduler0
            torch.cuda.empty_cache()
      
        for  batch in tqdm(train_dataloader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            BERTweet.zero_grad()
            outputs = BERTweet(b_input_ids, 
              token_type_ids=None, 
              attention_mask=b_input_mask, 
              labels=b_labels)

            loss = outputs[0]
            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(BERTweet.parameters(), 1.0)
            optimizer.step()
          
            if not frozen:
                scheduler.step()
            else:
                scheduler0.step()

          
        avg_train_loss = total_loss / len(train_dataloader)
        loss_values.append(avg_train_loss)
        print(" Average training loss: {0:.4f}".format(avg_train_loss))


        print("Running Validation...")
        t0 = time.time()
        BERTweet.eval()
        eval_loss = 0

        total_labels = []
        total_pre = []
      
        for batch in tqdm(valid_dataloader):

            batch = tuple(t.to(device) for t in batch)

            b_input_ids, b_input_mask,
            b_labels = batch

            with torch.no_grad():
                outputs = BERTweet(b_input_ids, 
                token_type_ids=None, 
                attention_mask=b_input_mask)
                logits = outputs[0]
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                logits = softmax(logits, axis=1)
                total_pre.extend(logits[:,1].flatten())
                total_labels.extend(label_ids.flatten())

        roc_auc = roc_auc_score(total_labels, total_pre)

        if roc_auc > roc_max:
            roc_max = roc_auc
            print("*"*100)
            torch.save(BERTweet.state_dict(), "path_to_folder_save_model_{}".format(fold))
print("Training complete!")

# TESTING

In [ ]:
path_test = "path_to_test"

In [ ]:
import pandas as pd
def build_dataset(path):
    df = pd.read_csv(path)
    df['post_message'] = df['post_message'].apply(preprocess)
    return list(df['post_message'])   

test_text = build_dataset(path_test)

In [ ]:
test_encodings = tokenizer(test_text, truncation=True, padding=True)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

test_ids = torch.tensor(test_encodings['input_ids'])
test_masks = torch.tensor(test_encodings['attention_mask'])

test_data = TensorDataset(test_ids, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=32)

In [ ]:
from scipy.special import softmax
from tqdm import tqdm
import random
import time
device = 'cuda'
test_df = pd.read_csv(path_test)
for i in range(6,7):
    print(i)
    config = RobertaConfig.from_pretrained(
    "vinai/phobert-base", from_tf=False, num_labels = 2, output_hidden_states=True,
    )
    BERTweet = RobertaTweetClassification.from_pretrained(
      "path-to_folder_saved_model".format(i),
      config=config

    )

    BERTweet.to('cuda')
    print("Running Test...")
    BERTweet.eval()
    eval_loss = 0

    total_pre = []

    for batch in tqdm(test_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask = batch
        with torch.no_grad():
            outputs = BERTweet(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            logits = softmax(logits, axis=1)
            total_pre.extend(logits[:,1].flatten())

    with open('path_to_test'.format(i), 'w') as f_w:
        for id, logit in zip(list(test_df['id']), total_pre):
            f_w.write(str(id) + ',' + str(logit) + '\n')

In [ ]:
var = {}
for i in range(6,7):
    datas= []
    with open ('/content/gdrive/MyDrive/file/results_{}.csv'.format(i), 'r') as f_w:
        for data in f_w:
            data = data.strip().split(",")[1]
            datas.append(float(data))
            var[str(i)] = datas

In [ ]:
pred = []
for i in range(len(var['9'])):
    data =var['6'][i]
    data = max(var['0'][i], var['1'][i], var['2'][i], var['3'][i], var['4'][i], var['5'][i], var['6'][i], var['7'][i], var['8'][i])
    pred.append(str(data))

In [ ]:
test_df = pd.read_csv(path_test)
with open('/content/gdrive/MyDrive/file/results.csv', 'w') as f_w:
    for id, logit in zip(list(test_df['id']), pred):
        f_w.write(str(id) + ',' + str(logit) + '\n')